In [1]:
import sys
sys.path.append("..")

## Falcon-7b

In [2]:
import copy
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationChain
from langchain import PromptTemplate, LLMChain
from IPython.display import display, Markdown

from src.llm import print_num_params

In [3]:
model_name = "tiiuae/falcon-7b-instruct"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto",
    trust_remote_code=True,
)
model = copy.deepcopy(base_model)

In [ ]:
fine_tuning_template = """
Human: {question}
AI: {best_answer}
"""
fine_tune_prompt = PromptTemplate.from_template(fine_tuning_template)

def tokenize(item):
    return tokenizer(
        fine_tune_prompt.format(
            question=item["question"], best_answer=item["best_answer"]
        ),
        padding=True,
        truncation=True,
    )

In [ ]:
from datasets import load_dataset

data = load_dataset("truthful_qa", "generation")
data = data["validation"].filter(lambda item: item["category"] == "Misconceptions")
train_dataset = data.map(tokenize)

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
    trust_remote_code=True,
)

In [ ]:
print_num_params(model)

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
print_num_params(model)

In [ ]:
output_dir = "fine_tuned"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=output_dir,
    max_steps=100,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
)
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

In [ ]:
from src.llm import get_chain

default_template = """
Human: {input} 
AI:"""

def compare_results(base_model, fine_tuned_model, item, template=None, verbose=False):
    if template is None:
        template = default_template
    base_chain = get_chain(base_model, tokenizer, template, verbose)
    fine_tuned_chain = get_chain(fine_tuned_model, tokenizer, template, verbose)
    base_res = base_chain.run(item["question"])
    fine_tuned_res = fine_tuned_chain.run(item["question"])
    display(
        Markdown(
            f"""
### question: 
{item['question']}
#### base_model:
{base_res}
#### fine_tuned_model:
{fine_tuned_res}
#### best answer:
{item['best_answer']}
#### source:
{item['source']}
"""
        )
    )

In [ ]:
compare_results(base_model, model, data[6])

## Phi-1.5

### Fine-tuning

#### Model and Tokenizer

In [1]:
import copy
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [2]:
model_name = "microsoft/phi-1_5"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    # load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True,
)
model = copy.deepcopy(base_model)

#### Dataset Preperation

In [5]:
from datasets import load_dataset

In [6]:
cutoff_len = 256

def generate_prompt(data_point):
  return f"""{data_point["Question"]}

Answer: {data_point["Answer"]}
  """.strip()


def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  result = tokenizer(full_prompt, padding='max_length', truncation=True, max_length=cutoff_len) # , return_tensors=None)
  result['data'] = full_prompt
  return result

In [7]:
dataset = load_dataset('json', data_files='../assets/qa_gpt4.json', split="train")
dataset = dataset.shuffle().map(generate_and_tokenize_prompt)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/131 [00:00<?, ? examples/s]

In [8]:
print(dataset[0].keys())
dataset[0]['data']
len(dataset[0]['input_ids'])

dict_keys(['Answer', 'Question', 'input_ids', 'attention_mask', 'data'])


256

#### Training

In [15]:
OUTPUT_DIR = "/root/hongyu/JupyterNotebooksFinetuning/models/phi1.5"
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=64,
    gradient_accumulation_steps=4,
    # warmup_steps=100,
    auto_find_batch_size=True,
    num_train_epochs=1,
    learning_rate=1e-6,  # 2e-5,
    weight_decay=0.1,
    fp16=False,
    # optim='adamw_torch',
    # bf16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    save_strategy='epoch',
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6
)

In [16]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False) #, return_tensors='pt')  #, pad_to_multiple_of=8),
)

In [17]:
model.config.use_cache = False
trainer.train()

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


Step,Training Loss
1,2.006600
2,1.909400
3,1.858300
4,1.815500


TrainOutput(global_step=4, training_loss=1.897437334060669, metrics={'train_runtime': 9.338, 'train_samples_per_second': 14.029, 'train_steps_per_second': 0.428, 'total_flos': 258227526696960.0, 'train_loss': 1.897437334060669, 'epoch': 0.94})

### Inference

In [21]:
from src.llm.phi import phi_1_5_inference

In [22]:
base_output = phi_1_5_inference(base_model, tokenizer, question=dataset[0]["Question"])

print("Question: " + dataset[0]["Question"])
print("#################################")
print("GPT-4: " + dataset[0]["Answer"])
print("#################################")
print("Base Model: " + base_output)

ft_output = phi_1_5_inference(model, tokenizer, question=dataset[0]["Question"])
print("#################################")
print("Fine-tuned Model: " + ft_output)

Question: Can you tell me about Jordan Grumet's presence in the FIRE world?
#################################
GPT-4: In the Financial Independence, Retire Early (FIRE) community, Jordan Grumet is known as Doc G. He is recognized not only for his work as a hospice doctor but also as a host of the Earn & Invest Podcast that discusses similar financial independence topics.
#################################
Base Model: Jordan Grumet is a renowned expert in the field of fire safety and prevention.
#################################
Fine-tuned Model: Jordan Grumet is a prominent figure in the Financial Independence, Retire Early (FIRE) community.
